### Load and analyze the dataset

In [1]:
from datasets import load_dataset

# Load the custom dataset (or it if it does not exist)
# Param 1: location of the dataset loader script
# Param 2: location of cache folder, where the dataset will be saved
dataset = load_dataset(r'C:\Users\Habram\Documents\thesis-masters\IstVoices_de\istvoices_dataset_de.py',
                       cache_dir=r'C:\Users\Habram\.cache')

c:\Users\Habram\Anaconda3\envs\LayoutXLM\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset istvoices_dataset_de (C:/Users/Habram/.cache/istvoices_dataset_de/default/0.0.0/1f9f58bf326613f8f47333b34087510d0faf21ae9d46bc895f367a6c1810165e)
100%|██████████| 2/2 [00:00<00:00, 266.94it/s]


In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'bboxes', 'ner_tags', 'image'],
        num_rows: 150
    })
    test: Dataset({
        features: ['id', 'tokens', 'bboxes', 'ner_tags', 'image'],
        num_rows: 50
    })
})

In [3]:
dataset['train'].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'bboxes': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-R_NAME', 'I-R_NAME', 'B-R_STREET', 'I-R_STREET', 'B-R_HOUSENUMBER', 'I-R_HOUSENUMBER', 'B-R_ZIP', 'I-R_ZIP', 'B-R_CITY', 'I-R_CITY', 'B-R_COUNTRY', 'I-R_COUNTRY', 'B-S_NAME', 'I-S_NAME', 'B-S_STREET', 'I-S_STREET', 'B-S_HOUSENUMBER', 'I-S_HOUSENUMBER', 'B-S_ZIP', 'I-S_ZIP', 'B-S_CITY', 'I-S_CITY', 'B-S_COUNTRY', 'I-S_COUNTRY', 'B-S_BANK', 'I-S_BANK', 'B-S_IBAN', 'I-S_IBAN', 'B-I_NUMBER', 'I-I_NUMBER', 'B-I_DATE', 'I-I_DATE', 'B-I_AMOUNT', 'I-I_AMOUNT'], id=None), length=-1, id=None),
 'image': Image(decode=True, id=None)}

### Prepare dataset for the model

In [4]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("microsoft/layoutxlm-base", apply_ocr=False)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LayoutXLMTokenizer'. 
The class this function is called from is 'LayoutLMv2TokenizerFast'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
from datasets.features import ClassLabel

features = dataset["train"].features
column_names = dataset["train"].column_names
image_column_name = "image"
text_column_name = "tokens"
boxes_column_name = "bboxes"
label_column_name = "ner_tags"

# Define the dictionaries which associate the labels with integer IDs
label_list = features[label_column_name].feature.names
id2label = {k: v for k,v in enumerate(label_list)}
label2id = {v: k for k,v in enumerate(label_list)}

In [6]:
from transformers import LayoutLMv2FeatureExtractor, LayoutXLMTokenizer

feature_extractor = LayoutLMv2FeatureExtractor(apply_ocr=False)
tokenizer = LayoutXLMTokenizer.from_pretrained("microsoft/layoutxlm-base")

c:\Users\Habram\Anaconda3\envs\LayoutXLM\lib\site-packages\transformers\models\layoutlmv2\feature_extraction_layoutlmv2.py:30: FutureWarning: The class LayoutLMv2FeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use LayoutLMv2ImageProcessor instead.
  warnings.warn(


In [22]:
# Define a function, which takes an example from the general-purpose dataset and processes it with the model's tokenizer.
# The result aligns with the expected format from the model.

def prepare_examples(examples):
  images = examples[image_column_name]
  words = examples[text_column_name]
  boxes = examples[boxes_column_name]
  word_labels = examples[label_column_name]

  encoding = tokenizer(words, boxes=boxes, word_labels=word_labels, truncation=True, padding="max_length", max_length=512)
  #print(type(encoding))
  img_features = feature_extractor(images).pixel_values

  encoding['pixel_values'] = img_features
  #print(type(encoding))
  return encoding

In [23]:
from datasets import Features, Sequence, Value, Array2D, Array3D

# Define the features of the model-specific dataset
features = Features({
    'pixel_values': Array3D(dtype="float32", shape=(3, 224, 224)),
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'attention_mask': Sequence(Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'labels': Sequence(feature=Value(dtype='int64')),
})


# Convert the general-purpose dataset into a model-specific dataset
train_dataset = dataset["train"].map(
    prepare_examples,
    batched=True,
    remove_columns=column_names,
    features=features,
)
test_dataset = dataset["test"].map(
    prepare_examples,
    batched=True,
    remove_columns=column_names,
    features=features,
)

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [24]:
train_dataset

Dataset({
    features: ['pixel_values', 'input_ids', 'attention_mask', 'bbox', 'labels'],
    num_rows: 150
})